## QXD0178 - Mineração de Dados
# Exercício de Classificação de Dados

# Análise de Classificadores para Autopercepção de Peso

### Objetivo

Este exercício foca na aplicação e avaliação de diferentes algoritmos de aprendizado de máquina para uma tarefa de classificação. Utilizando o conjunto de dados [Food choices: College students' food and cooking preferences](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv), o objetivo é prever a autopercepção de sobrepeso (`self_perception_overweight`) dos estudantes.

O processo inclui um pré-processamento detalhado dos dados, a divisão em conjuntos de treino e teste, e o treinamento de múltiplos classificadores (Naive Bayes, KNN, SVM, Decision Trees, Random Forest, e MLP). A performance dos modelos é comparada usando métricas padrão, validação cruzada e ajuste de hiperparâmetros.

## Implementação e Análise


### 1. Pré-processamento dos dados

O dataset "Food choices..." agrega 125 registros de estudantes universitários sobre seus hábitos alimentares. Os dados originais são brutos, contendo valores ausentes e inconsistências que necessitam de tratamento.

A etapa de pré-processamento envolveu os seguintes passos:

- **Tratamento de Valores Ausentes:**
  - Atributos numéricos com poucos dados faltantes (`calories_scone`, `tortilla_calories`) foram imputados usando a mediana.
  - Diversas colunas categóricas (como `cook`, `cuisine`, `employment`, `income`, `sports`, `self_perception_weight`, etc.) tiveram os valores ausentes preenchidos com a moda (valor mais frequente).
  - Para `type_sports`, sendo texto livre, os valores nulos foram substituídos pela string 'none'.

- **Correção de Inconsistências:**
  - O atributo `GPA` continha entradas não numéricas. Foi realizada uma conversão para tipo numérico, e os valores que falharam na conversão (tornaram-se `NaN`) foram preenchidos com a mediana.
  - A coluna `weight` também apresentava dados textuais. Foi aplicado um processo para extrair apenas os dígitos numéricos, e o tratamento de `NaN` seguiu a mesma lógica do `GPA` (imputação pela mediana).

- **Codificação (Encoding):**
  - Todos os atributos do tipo 'object' (categóricos textuais) foram convertidos em representações numéricas através do `LabelEncoder`, tornando-os compatíveis com os algoritmos de classificação.

- **Engenharia de Atributo (Alvo):**
  - O atributo alvo, `self_perception_overweight`, foi criado a partir da coluna `self_perception_weight`. Ele recebe o valor `True` se a percepção original era 4 ou 5, e `False` nos outros casos.
  - Após a criação da nova coluna, `self_perception_weight` foi descartada do conjunto de dados.

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Carga dos dados
df = pd.read_csv('https://raw.githubusercontent.com/She-Codes-Now/Intro-to-Data-Science-with-R/master/food_coded.csv')

# Tratamento de inconsistências e valores ausentes
df['GPA'] = pd.to_numeric(df['GPA'], errors='coerce')
df['GPA'].fillna(df['GPA'].median(), inplace=True)

df['weight'] = df['weight'].astype(str).str.extract(r'(\d+)').astype(float)
df['weight'].fillna(df['weight'].median(), inplace=True)

categorical_cols_with_nan = [
    'calories_day', 'comfort_food_reasons_coded', 'cook', 'cuisine',
    'diet_current_coded', 'eating_out', 'employment', 'ethnic_food',
    'father_education', 'healthy_feeling', 'income', 'mother_education',
    'self_perception_weight', 'sports', 'soup'
]
for col in categorical_cols_with_nan:
    df[col].fillna(df[col].mode()[0], inplace=True)

df['type_sports'].fillna('none', inplace=True)
df['calories_scone'].fillna(df['calories_scone'].median(), inplace=True)
df['tortilla_calories'].fillna(df['tortilla_calories'].median(), inplace=True)

# Codificação de variáveis categóricas
le = LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
    df[col] = le.fit_transform(df[col])

# Criação da coluna alvo
df['self_perception_overweight'] = df['self_perception_weight'].isin([4, 5])
df.drop('self_perception_weight', axis=1, inplace=True)

print("Pré-processamento concluído.")
df.head()

Pré-processamento concluído.


/tmp/ipython-input-2919372931.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['GPA'].fillna(df['GPA'].median(), inplace=True)
/tmp/ipython-input-2919372931.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

,GPA,Gender,breakfast,calories_chicken,calories_day,calories_scone,coffee,comfort_food,comfort_food_reasons,comfort_food_reasons_coded,...,sports,thai_food,tortilla_calories,turkey_calories,type_sports,veggies_day,vitamins,waffle_calories,weight,self_perception_overweight
0,2.400,2,1,430,3.0,315.0,1,110,104,9.0,...,1.0,1,1165.0,345,39,5,1,1315,187.0,False
1,3.654,1,1,610,3.0,420.0,2,83,57,1.0,...,1.0,2,725.0,690,1,4,2,900,155.0,False
2,3.300,1,1,720,4.0,420.0,2,98,100,1.0,...,2.0,5,1165.0,500,49,5,1,900,155.0,False
3,3.200,1,1,430,3.0,420.0,2,59,6,2.0,...,2.0,5,725.0,690,49,3,1,1315,240.0,True
4,3.500,1,1,720,2.0,420.0,2,34,61,1.0,...,1.0,4,940.0,500,27,4,2,760,190.0,True


### 2. Divisão do conjunto de dados

Os dados foram segmentados em conjuntos de treinamento (80% dos dados) e teste (20% restantes). A definição de um `random_state` fixo (42) assegura que a divisão seja sempre a mesma, permitindo a reprodutibilidade da análise.

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('self_perception_overweight', axis=1)
y = df['self_perception_overweight']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 3. Seleção de algoritmos de classificação

Para a tarefa de classificação, foram escolhidos os seguintes modelos:

- **Naive Bayes:** Um classificador probabilístico que aplica o teorema de Bayes, assumindo independência condicional entre os atributos.
- **k-Nearest Neighbors (KNN):** Um método não paramétrico (baseado em instância) que classifica um novo dado com base no voto majoritário de seus 'k' vizinhos mais próximos no espaço de atributos.
- **Support Vector Machine (SVM):** Busca o hiperplano ótimo que separa as classes com a maior margem possível. Foram testadas duas variações de kernel: 'linear' e 'RBF'.
- **Decision Trees:** Um modelo que cria uma estrutura de fluxo (árvore) onde decisões são tomadas em cada nó com base em atributos, levando a uma classificação nas folhas.
- **Random Forest:** Um classificador do tipo *ensemble* que constrói múltiplas árvores de decisão durante o treinamento e utiliza a moda de suas previsões para classificar.
- **Multilayer Perceptron (MLP):** Um modelo de rede neural artificial (feedforward) com camadas ocultas, projetado para aprender padrões não lineares complexos.

### 4. Treinamento e avaliação

In [7]:
# Importe as classes necessárias
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Assumindo que 'pd', 'X', 'y', 'X_train', 'y_train', 'X_test', 'y_test' já existem
import pandas as pd

classifiers = {
    'Naive Bayes': GaussianNB(),
    'k-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine (Linear)': SVC(kernel='linear'),
    'Support Vector Machine (RBF)': SVC(kernel='rbf'),
    'Decision Trees': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Multilayer Perceptron': MLPClassifier(max_iter=1000)
}

results = {}

# Crie um imputer.
# Estratégia pode ser 'mean' (média), 'median' (mediana) ou 'most_frequent' (moda)
imputer = SimpleImputer(strategy='mean')

for name, clf in classifiers.items():
    # Crie um Pipeline para cada classificador
    # 1º passo: 'imputer' (preenche NaNs)
    # 2º passo: 'classifier' (o seu modelo)
    pipeline = Pipeline(steps=[('imputer', imputer),
                             ('classifier', clf)])

    # 1. Treinamento e avaliação simples
    # Treine o pipeline (ele vai imputar X_train e depois treinar o clf)
    pipeline.fit(X_train, y_train)
    # Use o pipeline para prever (ele vai imputar X_test e depois prever)
    y_pred = pipeline.predict(X_test)

    # 2. Validação cruzada
    # Passe o *pipeline* inteiro para o cross_val_score.
    # Ele cuidará da imputação corretamente em cada "fold" da validação.
    # Use os dados originais X e y (com NaNs)
    cv_scores = cross_val_score(pipeline, X, y, cv=5)

    # Armazena os resultados (sem alteração aqui)
    results[name] = {
        'Acurácia': accuracy_score(y_test, y_pred),
        'Precisão': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted'),
        'Acurácia (CV)': cv_scores.mean()
    }

results_df = pd.DataFrame(results).T
print("--- Resultados ---")
print(results_df)

# --- Ajuste de hiperparâmetros (CORRIGIDO) ---

# Crie um pipeline *específico* para o GridSearchCV
# (É o mesmo imputer, mas o classificador que queremos otimizar)
rf_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                            ('classifier', RandomForestClassifier())])

# IMPORTANTE: Ao ajustar parâmetros de um pipeline, você deve
# prefixar o nome do parâmetro com o nome do passo no pipeline.
# Ex: 'n_estimators' vira 'classifier__n_estimators'
param_grid = {
    'classifier__n_estimators': [10, 50, 100],
    'classifier__max_depth': [None, 10, 20]
}

# Passe o *pipeline* para o GridSearchCV
grid_search = GridSearchCV(rf_pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train) # Agora isso funciona

best_rf = grid_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test)

print("\n--- Melhores Hiperparâmetros (Random Forest) ---")
print(grid_search.best_params_)
print(f"Acurácia (RF com ajuste): {accuracy_score(y_test, y_pred_best_rf)}")

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


--- Resultados ---
                                 Acurácia  Precisão  Recall  F1-Score  \
Naive Bayes                          0.60  0.600000    0.60  0.600000   
k-Nearest Neighbors                  0.64  0.570303    0.64  0.581259   
Support Vector Machine (Linear)      0.64  0.679740    0.64  0.651002   
Support Vector Machine (RBF)         0.68  0.462400    0.68  0.550476   
Decision Trees                       0.56  0.544762    0.56  0.551619   
Random Forest                        0.72  0.801667    0.72  0.635014   
Multilayer Perceptron                0.60  0.600000    0.60  0.600000   

                                 Acurácia (CV)  
Naive Bayes                              0.672  
k-Nearest Neighbors                      0.648  
Support Vector Machine (Linear)          0.616  
Support Vector Machine (RBF)             0.704  
Decision Trees                           0.648  
Random Forest                            0.728  
Multilayer Perceptron                    0.648  

---

# Análise de Resultados

## Resumo Geral

A análise comparativa de diversos algoritmos de machine learning indicou que o **Random Forest** foi o modelo com o melhor desempenho geral para este conjunto de dados. Ele obteve a maior acurácia tanto na divisão inicial de treino/teste quanto na validação cruzada (Cross-Validation), que é uma métrica mais robusta.

## Comparação de Desempenho (Validação Cruzada)

A Acurácia da Validação Cruzada (CV) é a métrica mais confiável para comparar o desempenho generalista dos modelos. Os resultados foram os seguintes:

| Modelo | Acurácia (CV) |
| :--- | :--- |
| **Random Forest** | **0.728** |
| Support Vector Machine (RBF) | 0.704 |
| Naive Bayes | 0.672 |
| K-Nearest Neighbors | 0.648 |
| Decision Trees | 0.648 |
| Multilayer Perceptron | 0.648 |
| Support Vector Machine (Linear) | 0.616 |

O **Random Forest** (72.8%) e o **SVM (RBF)** (70.4%) foram os únicos modelos a ultrapassar 70% de acurácia na validação cruzada, destacando-se como as melhores escolhas. Os modelos KNN, Decision Trees e MLP tiveram um desempenho idêntico (64.8%), enquanto o SVM Linear apresentou o pior resultado (61.6%).

## Análise do Modelo Vencedor (Random Forest)

O Random Forest não só teve a melhor acurácia em CV, mas também apresentou os melhores indicadores na divisão inicial de treino/teste:

* **Acurácia:** 0.72
* **Precisão:** 0.80 (A mais alta entre todos, indicando que suas previsões positivas são frequentemente corretas)
* **Recall:** 0.72
* **F1-Score:** 0.63

### Otimização de Hiperparâmetros

O modelo Random Forest passou por um processo de ajuste de hiperparâmetros, que identificou a seguinte configuração como a ideal:

* `classifier__max_depth`: `None` (As árvores são expandidas até que todas as folhas sejam puras)
* `classifier__n_estimators`: `50` (O modelo é composto por 50 árvores de decisão)

A acurácia final do modelo ajustado foi de **0.72**, confirmando o desempenho robusto observado na validação cruzada.

## Observações Importantes

* **Aviso de Métrica (`UndefinedMetricWarning`)**: O sistema emitiu um aviso informando que a **Precisão** é "ill-defined" (mal definida) e está sendo definida como 0.0 em pelo menos um "label" (classe).
* **Implicação**: Isso geralmente acontece quando não há nenhuma predição positiva para uma determinada classe. Esse cenário sugere um **forte desbalanceamento de classe** no conjunto de dados, onde o modelo falha em prever instâncias da classe minoritária. Isso explica por que alguns modelos, como o SVM (RBF), tiveram uma acurácia razoável (0.68) mas uma precisão muito baixa (0.46) na divisão inicial.

## Conclusão

O **Random Forest** é o modelo recomendado. No entanto, o aviso sobre a precisão sugere que, para melhorias futuras, deve-se investigar o desbalanceamento de classe. Técnicas como *oversampling* (ex: SMOTE) ou *undersampling*, ou o ajuste de pesos de classe (`class_weight`), podem ser necessárias para melhorar o F1-Score e o Recall das classes minoritárias.